In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import math

In [ ]:
df = pd.read_csv('df_clean.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['pageviews'] = df['pageviews'].astype(int)
df['newVisits'] = df['newVisits'].astype(int)
df['bounces'] = df['bounces'].astype(int)
df['hits'] = df['hits'].astype(int)
df['sessionQualityDim'] = df['sessionQualityDim'].astype(int)
df['timeOnSite'] = df['timeOnSite'].astype(int)
df['transactions'] = df['transactions'].astype(int)
df['totalTransactionRevenue'] = df['totalTransactionRevenue'].astype(int)
df['fullVisitorId'] = df['fullVisitorId'].astype(str)

In [ ]:
df.dtypes

In [ ]:
# All histograms
df.hist(figsize = (15,10))
plt.subplots_adjust(hspace=0.5);

In [ ]:
plt.figure(figsize=(12,10))
print('Correlation between various features')
p=sns.heatmap(df.corr(), annot=True,cmap ='Blues')

In [ ]:
df_none_zero = df[df['totalTransactionRevenue'] > 0]

In [ ]:
df['totalTransactionRevenue'].describe()

In [ ]:
df_none_zero['totalTransactionRevenue'].describe()

In [ ]:
# Total Transaction Revenue from None-Zero Transactions
w = 50000000
n = math.ceil((df['totalTransactionRevenue'].max() - df['totalTransactionRevenue'].min())/w)
plt.figure(figsize=(10, 8))
plt.hist(df_none_zero['totalTransactionRevenue'], bins = n)
plt.title('Total Transaction Revenue from None-Zero Transactions')
plt.xlabel('Total Transaction Revenue')
plt.ylabel('Transactions');

In [ ]:
# Total Transactions count from all Transactions
plt.figure(figsize=(10,8))
df.groupby('transactions')['totalTransactionRevenue'].count().sort_values(ascending=False).plot(kind='bar')
plt.ylabel('Total Transaction Count')
plt.xlabel('Transactions');

In [ ]:
# Total Transaction Revenue from all Transactions
plt.figure(figsize=(10,8))
df.groupby('transactions')['totalTransactionRevenue'].sum().sort_values(ascending=False).plot(kind='bar')
plt.ylabel('Total Transaction Revenue')
plt.xlabel('Number of Transactions');

In [ ]:
# Transaction revenue and visitnumber scatterplot sorted by transactions
sns.relplot(data=df, x='totalTransactionRevenue', y='visitNumber', hue='transactions', size=3, aspect=1.5);

In [ ]:
# Transaction revenue and visitStartTime scatterplot sorted by transactions
sns.relplot(data=df, x='totalTransactionRevenue', y='visitStartTime', hue='transactions', size=3, aspect=1.5);

In [ ]:
# Transaction revenue and timeOnSite scatterplot sorted by transactions
sns.relplot(data=df, x='totalTransactionRevenue', y='timeOnSite', hue='transactions', size=3, aspect=1.5);

In [ ]:
# Transaction count for all Transactions by browser
df_grouped = df.groupby(['browser', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero transactions revenue by Browser
df_grouped = df_none_zero.groupby(['browser', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by browser
df_grouped = df.groupby(['browser', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by operating system
df_grouped = df.groupby(['operatingSystem', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by operating system
df_grouped = df_none_zero.groupby(['operatingSystem', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by browser
df_grouped = df.groupby(['operatingSystem', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'isMobile'
df_grouped = df.groupby(['isMobile', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'isMobile'
df_grouped = df_none_zero.groupby(['isMobile', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'isMobile'
df_grouped = df.groupby(['isMobile', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'deviceCategory'
df_grouped = df.groupby(['deviceCategory', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'deviceCategory'
df_grouped = df_none_zero.groupby(['deviceCategory', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'deviceCategory'
df_grouped = df.groupby(['deviceCategory', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'subContinent'
df_grouped = df.groupby(['subContinent', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8),  stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'subContinent'
df_grouped = df_none_zero.groupby(['subContinent', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'subContinent'
df_grouped = df.groupby(['subContinent', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue and pageviews scartterplot sorted by transactions
sns.relplot(data=df, x='totalTransactionRevenue', y='pageviews', hue='transactions', size=3, aspect=1.5);

In [ ]:
# Transaction count for all Transactions by 'bounces'
df_grouped = df.groupby(['bounces', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'bounces'
df_grouped = df_none_zero.groupby(['bounces', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'bounces'
df_grouped = df.groupby(['bounces', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'newVisits'
df_grouped = df.groupby(['newVisits', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by ''newVisits'
df_grouped = df_none_zero.groupby(['newVisits', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'newVisits'
df_grouped = df.groupby(['newVisits', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue and pageviews scatterpolt sorted by transactions
sns.relplot(data=df, x='totalTransactionRevenue', y='sessionQualityDim', hue='transactions', size=3, aspect=1.5);

In [ ]:
# Transaction count for all Transactions by 'weekday'
df_grouped = df.groupby(['weekday', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by ''weekday'
df_grouped = df_none_zero.groupby(['weekday', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'weekday'
df_grouped = df.groupby(['weekday', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'day'
df_grouped = df.groupby(['day', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'day'
df_grouped = df_none_zero.groupby(['day', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'day'
df_grouped = df.groupby(['day', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'month'
df_grouped = df.groupby(['month', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'month'
df_grouped = df_none_zero.groupby(['month', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction revenue by 'month'
df_grouped = df.groupby(['month', 'transactions'])['totalTransactionRevenue'].sum().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for all Transactions by 'year'
df_grouped = df.groupby(['year', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# Transaction count for none zero Transactions by 'year'
df_grouped = df_none_zero.groupby(['year', 'transactions'])['totalTransactionRevenue'].count().unstack()
df_grouped[0]= 0
df_grouped = df_grouped[[0, 1, 2, 3, 4, 5, 8]]
df_grouped.plot(kind='barh', figsize=(10, 8), stacked=True);

In [ ]:
# all transactions
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df, x = 'weekday', y = 'totalTransactionRevenue')

In [ ]:
# none zero transactions revenue
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df_none_zero, x = 'weekday', y = 'totalTransactionRevenue')

In [ ]:
# all transactions
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df, x = 'year', y = 'totalTransactionRevenue')

In [ ]:
# none zero transactions revenue
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df_none_zero, x = 'year', y = 'totalTransactionRevenue')

In [ ]:
# all transactions
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df, x = 'transactions', y = 'totalTransactionRevenue')

In [ ]:
# none zero transactions revenue
plt.figure(figsize = [10, 8]) 
sns.violinplot(data = df_none_zero, x = 'transactions', y = 'totalTransactionRevenue')

In [ ]:
df.to_csv('df.csv', index=False)